In [ ]:
import yfinance as yf
import os
import pandas as pd
# import tensorflow as tf

In [ ]:
btc_ticker = yf.Ticker("BTC-USD")


In [ ]:
if os.path.exists("D:\LEARNING AI\TỰ HỌC\DATA\ibtc.csv"):
    btc = pd.read_csv("D:\LEARNING AI\TỰ HỌC\DATA\ibtc.csv", index_col=0)
else:
    btc = btc_ticker.history(period="max")
    btc.to_csv("D:\LEARNING AI\TỰ HỌC\DATA\ibtc.csv")
# btc = btc_ticker.history(period="max")
# btc.to_csv("D:\LEARNING AI\TỰ HỌC\DATA\ibtc.csv")

In [ ]:
btc

In [ ]:
btc.index = pd.to_datetime(btc.index)

In [ ]:
del btc["Dividends"]
del btc["Stock Splits"]

In [ ]:
btc.columns = [c.lower() for c in btc.columns]

In [ ]:
btc.plot.line(y="close", use_index=True)

In [ ]:
wiki = pd.read_csv("D:\LEARNING AI\TỰ HỌC\DATA\wikipedia_edits.csv", index_col=0, parse_dates=True)

In [ ]:
wiki

In [ ]:
btc = btc.merge(wiki, left_index=True, right_index=True)

In [ ]:
btc["tomorrow"] = btc["close"].shift(-1)

In [ ]:
btc["target"] = (btc["tomorrow"] > btc["close"]).astype(int)

In [ ]:
btc["target"].value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=50, random_state=1)

train = btc.iloc[:-1000]
test = btc.iloc[-1000:]

predictors = ["close", "volume", "open", "high", "low", "edit_count", "sentiment", "neg_sentiment"]
model.fit(train[predictors], train["target"])

In [ ]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
precision_score(test["target"], preds)

In [ ]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [ ]:
def backtest(data, model, predictors, start=1095, step=150):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state=1, learning_rate=.1, n_estimators=1000)
predictions = backtest(btc, model, predictors)

In [ ]:
predictions["predictions"].value_counts()

In [ ]:
precision_score(predictions["target"], predictions["predictions"])

In [ ]:
def compute_rolling(btc):
    horizons = [2,7,60,365]
    new_predictors = ["close", "sentiment", "neg_sentiment"]

    for horizon in horizons:
        rolling_averages = btc.rolling(horizon, min_periods=1).mean()

        ratio_column = f"close_ratio_{horizon}"
        btc[ratio_column] = btc["close"] / rolling_averages["close"]
        
        edit_column = f"edit_{horizon}"
        btc[edit_column] = rolling_averages["edit_count"]

        rolling = btc.rolling(horizon, closed='left', min_periods=1).mean()
        trend_column = f"trend_{horizon}"
        btc[trend_column] = rolling["target"]

        new_predictors+= [ratio_column, trend_column, edit_column]
    return btc, new_predictors

In [ ]:
btc

In [ ]:
btc, new_predictors = compute_rolling(btc.copy())

In [ ]:
predictions = backtest(btc, model, new_predictors)

In [ ]:
precision_score(predictions["target"], predictions["predictions"])

In [ ]:
predictions